In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geopy.distance import distance
import data_loaders


In [60]:
carnegie=data_loaders.gimmeCarnegieLimtedWithLocations()
ga_outcomes=data_loaders.gimmeGA()
county_income=data_loaders.gimmeCountyIncomes()
county_pop= pd.read_csv('data/uscounties.csv')

In [19]:
ga_pop=county_pop.loc[county_pop['state_id']=='GA'].reset_index()[['county','lat','lng','population']]
ga_income=county_income.loc[county_income['State_Abbreviation']=='GA'].reset_index()

In [ ]:
# Testing geopy
coord1 = (carnegie.iloc[1].latitude,carnegie.iloc[1].longitude)
coord2 = (ga_pop.iloc[1].lat,ga_pop.iloc[1].lng)
print(distance(coord1,coord2).miles)

163.27496179983422


These district are double counties and excluded

In [62]:
exceptions=ga_outcomes[ga_outcomes['County'].isin(county_pop['county'].values)==False]['County'].drop_duplicates()
print(exceptions)

144     Savannah-Chatham
1004    Griffin-Spalding
1058     Thomaston-Upson
Name: County, dtype: object


In [71]:
ga_outcomes_no_exceptions=ga_outcomes[ga_outcomes['County'].isin(county_pop['county'].values)==True]

In [33]:
carnegie_full = pd.read_excel('data/CCIHE2021-PublicData.xlsx',sheet_name='Data')
carnegie['stem_rsd'] = carnegie_full['stem_rsd']
carnegie['anenr1920'] = carnegie_full['anenr1920']

In [37]:
def closest_five(carnegie_univ_data,lat,long):

    # carnegie_univ_data is meant to be sublist/subdataframe of carnegie dataset (with location)
    # lat is meant to be the latitude of the location (usually county)
    # long is meant to be the longitude of the location
    # Returns the average distance to the five closest universities from the supplied list.

    univ_distance = {'unitid':[],'distance':[]}
    for i in carnegie_univ_data.index:
        univ_distance['unitid']=univ_distance['unitid']+[carnegie_univ_data.unitid[i]]
        univ_distance['distance']=univ_distance['distance']+[distance((lat,long),(carnegie_univ_data.latitude[i],carnegie_univ_data.longitude[i])).km]
    univ_distance = pd.DataFrame(univ_distance)
    closest_five = univ_distance.sort_values(by = 'distance')[:5]
    return np.mean(closest_five['distance'].values)


In [ ]:
ga_pop['closest 5 R1-2']=ga_pop.apply(lambda x: closest_five(carnegie[carnegie['basic2021'].isin([15,16])],x.lat,x.lng),axis=1)
ga_pop['closest 5 Public']=ga_pop.apply(lambda x: closest_five(carnegie[carnegie['control']==1],x.lat,x.lng),axis=1)
ga_pop['closest 5 Private Non-Profit']=ga_pop.apply(lambda x: closest_five(carnegie[carnegie['control']==2],x.lat,x.lng),axis=1)
ga_pop['closest 5 Landgrant']=ga_pop.apply(lambda x: closest_five(carnegie[carnegie['landgrnt']==1],x.lat,x.lng),axis=1)
ga_pop['closest_five_stem_avg'] = ga_pop.apply(lambda x: closest_five(carnegie[carnegie['stem_rsd'] > 0],x.lat, x.lng), axis=1)


In [41]:
def closest_five_enrollment(carnegie_univ_data,lat,long):

    # carnegie_univ_data is meant to be sublist/subdataframe of carnegie dataset (with location and anenr1920)
    # lat is meant to be the latitude of the location (usually county)
    # long is meant to be the longitude of the location
    # Returns the average annual enrollment of the five closest universities from the supplied list.

    univ_enrollment = {'unitid':[],'distance':[],'enrollment':[]}
    for i in carnegie_univ_data.index:
        univ_enrollment['unitid']=univ_enrollment['unitid']+[carnegie_univ_data.unitid[i]]
        univ_enrollment['distance']=univ_enrollment['distance']+[distance((lat,long),(carnegie_univ_data.latitude[i],carnegie_univ_data.longitude[i])).km]
        univ_enrollment['enrollment'] = univ_enrollment['enrollment'] + [carnegie_univ_data['anenr1920'][i]]
    univ_enrollment = pd.DataFrame(univ_enrollment)
    closest_five = univ_enrollment.sort_values(by = 'distance')[:5]
    return np.mean(closest_five['enrollment'].values)

def closest_five_rooms(carnegie_univ_data,lat,long):
        # carnegie_univ_data is meant to be sublist/subdataframe of carnegie dataset (with location and anenr1920)
    # lat is meant to be the latitude of the location (usually county)
    # long is meant to be the longitude of the location
    # Returns the average dorm rooms of the five closest universities from the supplied list.

    univ_enrollment = {'unitid':[],'distance':[],'rooms':[]}
    for i in carnegie_univ_data.index:
        univ_enrollment['unitid']=univ_enrollment['unitid']+[carnegie_univ_data.unitid[i]]
        univ_enrollment['distance']=univ_enrollment['distance']+[distance((lat,long),(carnegie_univ_data.latitude[i],carnegie_univ_data.longitude[i])).km]
        univ_enrollment['rooms'] = univ_enrollment['rooms'] + [carnegie_univ_data['rooms'][i]]
    univ_enrollment = pd.DataFrame(univ_enrollment)
    closest_five = univ_enrollment.sort_values(by = 'distance')[:5]
    return np.mean(closest_five['rooms'].values)



In [42]:
ga_pop['closest_five_avg_enrollment_r1r2']=ga_pop.apply(lambda x: closest_five_enrollment(carnegie[carnegie['basic2021'].isin([15,16])],x.lat,x.lng),axis=1)
ga_pop['closest_five_avg_enrollment_public']=ga_pop.apply(lambda x: closest_five_enrollment(carnegie[carnegie['control']==1],x.lat,x.lng),axis=1)
ga_pop['closest_five_avg_enrollment_private_nfp']=ga_pop.apply(lambda x: closest_five_enrollment(carnegie[carnegie['control']==2],x.lat,x.lng),axis=1)
ga_pop['closest_five_avg_enrollment_landgrnt']=ga_pop.apply(lambda x: closest_five_enrollment(carnegie[carnegie['landgrnt']==1],x.lat,x.lng),axis=1)
ga_pop['closest_five_avg_enrollment_stem'] = ga_pop.apply(lambda x: closest_five_enrollment(carnegie[carnegie['stem_rsd'] > 0],x.lat, x.lng), axis=1)


In [43]:
ga_pop['closest_five_avg_dormrooms_r1r2']=ga_pop.apply(lambda x: closest_five_rooms(carnegie[carnegie['basic2021'].isin([15,16])],x.lat,x.lng),axis=1)
ga_pop['closest_five_avg_dormrooms_public']=ga_pop.apply(lambda x: closest_five_rooms(carnegie[carnegie['control']==1],x.lat,x.lng),axis=1)
ga_pop['closest_five_avg_dormrooms_private_nfp']=ga_pop.apply(lambda x: closest_five_rooms(carnegie[carnegie['control']==2],x.lat,x.lng),axis=1)
ga_pop['closest_five_avg_dormrooms_landgrnt']=ga_pop.apply(lambda x: closest_five_rooms(carnegie[carnegie['landgrnt']==1],x.lat,x.lng),axis=1)
ga_pop['closest_five_avg_dormrooms_stem'] = ga_pop.apply(lambda x: closest_five_rooms(carnegie[carnegie['stem_rsd'] > 0],x.lat, x.lng), axis=1)


In [46]:
renamer={'closest 5 R1-2':'closest_five_r1r2_avg',
       'closest 5 Public':'closest_five_public_avg', 'closest 5 Private Non-Profit':'closest_five_private_nfp_avg',
       'closest 5 Landgrant':'closest_five_landgrnt_avg'}
ga_pop=ga_pop.rename(columns=renamer)

In [ ]:
ga_income_combiner=ga_income.rename(columns={'County':'county','Year':'year'})[['county','2018','2019','2020','2021','2022']]
ga_income_combiner=ga_income_combiner.melt(id_vars=['county'],value_vars=['2018','2019','2020','2021','2022'],var_name='year',value_name='income')


In [81]:
ga_data=pd.merge(ga_pop,ga_income_combiner,on='county')
ga_data.head()

,county,lat,lng,population,closest_five_r1r2_avg,closest_five_public_avg,closest_five_private_nfp_avg,closest_five_landgrnt_avg,closest_five_stem_avg,closest_five_avg_enrollment_r1r2,...,closest_five_avg_enrollment_private_nfp,closest_five_avg_enrollment_landgrnt,closest_five_avg_enrollment_stem,closest_five_avg_dormrooms_r1r2,closest_five_avg_dormrooms_public,closest_five_avg_dormrooms_private_nfp,closest_five_avg_dormrooms_landgrnt,closest_five_avg_dormrooms_stem,year,income
0,Fulton,33.7902,-84.467,1061944,12.95867,9.01348,6.913755,156.86541,27.635967,29745.0,...,1952.6,22160.2,29515.6,5252.0,2842.0,1019.6,5104.8,6238.8,2018,87609
1,Fulton,33.7902,-84.467,1061944,12.95867,9.01348,6.913755,156.86541,27.635967,29745.0,...,1952.6,22160.2,29515.6,5252.0,2842.0,1019.6,5104.8,6238.8,2019,93307
2,Fulton,33.7902,-84.467,1061944,12.95867,9.01348,6.913755,156.86541,27.635967,29745.0,...,1952.6,22160.2,29515.6,5252.0,2842.0,1019.6,5104.8,6238.8,2020,92999
3,Fulton,33.7902,-84.467,1061944,12.95867,9.01348,6.913755,156.86541,27.635967,29745.0,...,1952.6,22160.2,29515.6,5252.0,2842.0,1019.6,5104.8,6238.8,2021,101688
4,Fulton,33.7902,-84.467,1061944,12.95867,9.01348,6.913755,156.86541,27.635967,29745.0,...,1952.6,22160.2,29515.6,5252.0,2842.0,1019.6,5104.8,6238.8,2022,100614


In [76]:
ga_outcomes_no_exceptions.eval('Year=Year-1',inplace=True)

In [109]:
ga_outcomes_fin=pd.merge(ga_outcomes_no_exceptions.rename(columns={'County':'county'}),ga_data,on='county')

In [110]:
ga_outcomes_fin.head()

,SCHOOL_DSTRCT_NM,TEST_CMPNT_TYP_NM,NUMBER_TESTS_TAKEN,NOTESTS_3ORHIGHER,Year,Pass Rate,county,lat,lng,population,...,closest_five_avg_enrollment_private_nfp,closest_five_avg_enrollment_landgrnt,closest_five_avg_enrollment_stem,closest_five_avg_dormrooms_r1r2,closest_five_avg_dormrooms_public,closest_five_avg_dormrooms_private_nfp,closest_five_avg_dormrooms_landgrnt,closest_five_avg_dormrooms_stem,year,income
0,Appling County,ALL Subjects,92.0,37.0,2018,0.402174,Appling,31.7492,-82.2889,18441,...,5413.8,23627.6,27405.6,4776.6,1296.0,1865.0,5162.2,4772.0,2018,33662
1,Appling County,ALL Subjects,92.0,37.0,2018,0.402174,Appling,31.7492,-82.2889,18441,...,5413.8,23627.6,27405.6,4776.6,1296.0,1865.0,5162.2,4772.0,2019,35609
2,Appling County,ALL Subjects,92.0,37.0,2018,0.402174,Appling,31.7492,-82.2889,18441,...,5413.8,23627.6,27405.6,4776.6,1296.0,1865.0,5162.2,4772.0,2020,39134
3,Appling County,ALL Subjects,92.0,37.0,2018,0.402174,Appling,31.7492,-82.2889,18441,...,5413.8,23627.6,27405.6,4776.6,1296.0,1865.0,5162.2,4772.0,2021,42693
4,Appling County,ALL Subjects,92.0,37.0,2018,0.402174,Appling,31.7492,-82.2889,18441,...,5413.8,23627.6,27405.6,4776.6,1296.0,1865.0,5162.2,4772.0,2022,43602


In [111]:
#COUNTY,STUDENTS_TESTED,EXAM_COUNT,EXAMS_3_OR_ABOVE,PERCENT_3_OR_ABOVE,Year,Latitude,Longitude,population,per_capita_income,closest_five_r1r2_avg,closest_five_public_avg,closest_five_private_nfp_avg,closest_five_landgrnt_avg,closest_five_stem_avg,closest_five_avg_enrollment_r1r2,closest_five_avg_enrollment_public,closest_five_avg_enrollment_private_nfp,closest_five_avg_enrollment_landgrnt,closest_five_avg_enrollment_stem,closest_five_avg_dormrooms_r1r2,closest_five_avg_dormrooms_public,closest_five_avg_dormrooms_private_nfp,closest_five_avg_dormrooms_landgrant,closest_five_avg_dormrooms_stem
#['SCHOOL_DSTRCT_NM', 'TEST_CMPNT_TYP_NM', 'NUMBER_TESTS_TAKEN','NOTESTS_3ORHIGHER', 'Year', 'Pass Rate', 'county', 'lat', 'lng','population', 'closest_five_r1r2_avg', 'closest_five_public_avg','closest_five_private_nfp_avg', 'closest_five_landgrnt_avg','closest_five_stem_avg', 'closest_five_avg_enrollment_r1r2','closest_five_avg_enrollment_public','closest_five_avg_enrollment_private_nfp','closest_five_avg_enrollment_landgrnt','closest_five_avg_enrollment_stem', 'closest_five_avg_dormrooms_r1r2','closest_five_avg_dormrooms_public','closest_five_avg_dormrooms_private_nfp','closest_five_avg_dormrooms_landgrnt','closest_five_avg_dormrooms_stem', 'year', 'income']
ga_outcomes_fin=ga_outcomes_fin.rename(columns={'NUMBER_TESTS_TAKEN':'EXAM_COUNT','NOTESTS_3ORHIGHER':'EXAMS_3_OR_ABOVE','lat':'Latitude','lng':'Longitude','income':'per_capita_income','county':'COUNTY','Pass Rate':'PERCENT_3_OR_ABOVE'})

In [112]:
ga_outcomes_fin.eval('PERCENT_3_OR_ABOVE=PERCENT_3_OR_ABOVE*100',inplace=True)


In [113]:
ga_outcomes_fin=ga_outcomes_fin.loc[ga_outcomes_fin['TEST_CMPNT_TYP_NM']=='ALL Subjects']

In [20]:
from sklearn.model_selection import train_test_split
training, testing = train_test_split(ga_outcomes_fin, test_size = 0.2, random_state = 226)

In [21]:
training.to_pickle('data/GA_pickled/training.pkl')
testing.to_pickle('data/GA_pickled/testing.pkl')

In [19]:
ga_outcomes_fin.to_pickle('data/GA_pickled/ga_outcomes_full_data.pkl')